### 接口: 从协议到抽象基类

* .[猴子补丁](#猴子补丁)
* .[抽象基类](#abc)
    - .[静态接口](#check)
    - .[虚拟子类](#虚拟子类)
    - .[mro](#mro)

#### 猴子补丁

动态得定义类的属性,比如为了使得FrenchDeck有shuffle功能,定义\_\_setitem\_\_来继承可变序列的属性

In [38]:
class FrenchDeck:
    suits = 'spade heart diamond club'.split()
    ranks = list(range(2, 11)) + list('JQKA')
    def __init__(self):
        self._cards = [(rank, suit)   for suit in self.suits
                                       for rank in self.ranks]
        
    def __getitem__(self, pos):
        return self._cards[pos]
    
    def __len__(self):
        return len(self._cards)

In [39]:
def set_item(fd: FrenchDeck, pos: int, val):
    fd._cards[pos] = val 
    

FrenchDeck.__setitem__ = set_item

In [40]:
fd = FrenchDeck()
fd[:5]

[(2, 'spade'), (3, 'spade'), (4, 'spade'), (5, 'spade'), (6, 'spade')]

In [41]:
fd[:50:13]

[(2, 'spade'), (2, 'heart'), (2, 'diamond'), (2, 'club')]

In [42]:
from random import shuffle
shuffle(fd)
fd[:50:12]

[(7, 'heart'),
 ('Q', 'diamond'),
 ('K', 'diamond'),
 (7, 'diamond'),
 (5, 'spade')]

### 自定义抽象基类 <a id='abc'></a>


抽象基类Abstract Base Classes,主要是用于制定一套接口规范.

用abstractmethod来定义接口,子类必须包含相关方法.这些是抽象基类的静态接口.
同时也有动态特性: 虚拟子类

In [59]:
import abc


class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterables):
        """从可迭代对象中添加元素"""
    
    @abc.abstractmethod
    def pick(self):
        """随机删除元素,然后将其返回
        
        如果实例为空,抛出LookupError
        """
        
    def loaded(self):
        """如果至少有一个元素,返回True"""
        return bool(self.inspect())
    
    def inspect(self):
        """返回一个有序元祖,由当前元素组成"""
        items = []
        while True:
            try:
                item.append(self.pick())
            except LookupError:
                break 
        self.load(items)
        return tuple(sorted(items))

####  静态接口 <a id='check'></a>

实例化时检查子类是否符合抽象基类的接口

In [60]:
class Fake(Tombola):
    def pick(self):
        return 13
    
    
fake = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [66]:
import random 


class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()
  


haha = BingoCage(list('abc'))
print(haha())
haha.pick()

b


'c'

In [67]:
class LotteryBlower(Tombola):
    def __init__(self, iterables):
        self._balls = list(iterables)
        
    def load(self, iterables):
        self._balls.extend(iterables)
        
    def pick(self):
        try:
            pos = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(pos)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))
    
    
xixi = LotteryBlower(range(3))
print(issubclass(LotteryBlower, Tombola))
xixi.pick()

True


2

#### 虚拟子类

在子类上加装饰器
不继承基类接口,也不检查是否符合接口.但是issubclass和isinstance能识别

In [81]:
@Tombola.register
class TomboList(list):
    def pick(self):
        if self:
            pos = random.randrange(len(self))
            return self.pop(pos)
        else:
            raise LookupError('pick from empty TomboList')
        
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    

t = TomboList(range(10))
isinstance(t, Tombola)

True

#### mro

Method Resolution Order,按顺序列出类及其超类

In [70]:
TomboList.__mro__

(__main__.TomboList, list, object)

In [72]:
Tombola.__subclasses__()

[__main__.Fake, __main__.BingoCage, __main__.LotteryBlower]

In [82]:
Tombola._abc_